In [119]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.23.2', 'qiskit-aer': '0.12.0', 'qiskit-ignis': None, 'qiskit-ibmq-provider': '0.20.2', 'qiskit': '0.42.0', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [120]:
from qiskit import QuantumCircuit
# Create a circuit with a register of three qubits
circ = QuantumCircuit(1)
# H gate on qubit 0, putting this qubit in a superposition of |0> + |1>.
circ.h(0)
circ.x( 0)
circ.draw('mpl')
out_circ = qiskit.compiler.transpile(circ, unitary_synthesis_method="sk") 
out_circ.draw()

┌─────────────┐
q: ┤ U3(π/2,0,0) ├
   └─────────────┘

In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit
from qiskit.transpiler.passes.synthesis import SolovayKitaev
from qiskit.quantum_info import Operator

# create a basic circcuit with a single X gate on one qubit 
circuit = QuantumCircuit(1)
circuit.x(0)
print("Original circuit:")
print(circuit.draw())

# compile using the Solovay-Kitaev theorem built-in in Qiskit 
skd = SolovayKitaev(recursion_degree=2)
discretized = skd(circuit)
print("Discretized circuit:")
print(discretized.draw())
print("Error:", np.linalg.norm(Operator(circuit).data - Operator(discretized).data))

# why is the error so large? It's because there is a phase difference. See the next example for reference. 

Original circuit:
   ┌───┐
q: ┤ X ├
   └───┘
Discretized circuit:
global phase: π
   ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐
q: ┤ H ├┤ T ├┤ T ├┤ T ├┤ T ├┤ H ├
   └───┘└───┘└───┘└───┘└───┘└───┘
Error: 2.8284271247461894


In [2]:
from qiskit.synthesis import generate_basic_approximations
from qiskit.transpiler.passes import SolovayKitaev

# same as the previous cell except that now our basis (hardware instructions) are the Z and H (Hadamard) gate
basis = ["z", "h"]
approx = generate_basic_approximations(basis, depth=3)
skd = SolovayKitaev(recursion_degree=4, basic_approximations=approx)
discretized = skd(circuit)
for gate in discretized.data:
    print('\ngate name:', gate[0].name)
    print('qubit(s) acted on:', gate[1])
    print('other paramters (such as angles):', gate[0].params)

# Now compile the circuit again with Solovay-Kitaev theorem
print("Discretized circuit:")
print(discretized.draw())

# If we remove the phase of the of the output from the discretized circuit, the error becomes very small
print("Error:", np.linalg.norm(Operator(circuit).data - Operator(discretized).data*np.exp(np.pi*1.j)))

Discretized circuit:
global phase: π
   ┌───┐┌───┐┌───┐
q: ┤ H ├┤ Z ├┤ H ├
   └───┘└───┘└───┘
Error: 4.675343792338431e-16


In [8]:
for gate in discretized.data:
    print('\ngate name:', gate[0].name)
    print('qubit(s) acted on:', gate[1])
    print('other paramters (such as angles):', gate[0].params)


gate name: h
qubit(s) acted on: [Qubit(QuantumRegister(1, 'q'), 0)]
other paramters (such as angles): []

gate name: z
qubit(s) acted on: [Qubit(QuantumRegister(1, 'q'), 0)]
other paramters (such as angles): []

gate name: h
qubit(s) acted on: [Qubit(QuantumRegister(1, 'q'), 0)]
other paramters (such as angles): []


In [3]:
from qiskit.circuit.library import RXGate, TGate, HGate, XGate, ZGate
U = XGate().to_matrix()
A = HGate().to_matrix() @ ZGate().to_matrix() @ HGate().to_matrix()

In [4]:
np.linalg.norm(U-A)

3.1560822113208575e-16

In [5]:
Operator(discretized)

Operator([[ 0.+0.0000000e+00j, -1.+1.2246468e-16j],
          [-1.+1.2246468e-16j,  0.+0.0000000e+00j]],
         input_dims=(2,), output_dims=(2,))

In [10]:
from qiskit import QuantumCircuit
from qiskit.extensions import UnitaryGate

gate = UnitaryGate(A)
print(A)
circuit = QuantumCircuit(1)
circuit.append(gate, [0] )


[[-2.23711432e-17+0.j  1.00000000e+00+0.j]
 [ 1.00000000e+00+0.j -2.23711432e-17+0.j]]
